In [132]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import kneighbors_graph
from sklearn import cluster
import scipy
from scipy.linalg import eig

# Prova de Algebra Linear 

### Dataset escolhido: twitter-airline-sentiment disponivel no Kaggle
https://www.kaggle.com/crowdflower/twitter-airline-sentiment#Tweets.csv

O dataset foi escolhido pois ele possui texto e classificação quanto ao sentimento de cada tweet. O objetivo aqui será tentar aplicar as técnicas de Spectral Clustering para agrupar corretamente os tweets de acordo com seu conteúdo semântico

Esse notebook apresenta os resultados obtidos utilizando extração de atributos com a tecnica Tf-Idf, onde cada atributo representa a frequência de ocorrência de palavras em cada documento. As configurações de hiperparâmetros dos três métodos de Spectral Clustering foram testadas exaustivamente e os melhores resultados são apresentados e comparandos com a técnica mais básica de clustering, o kmeans

In [2]:
df = pd.read_csv("./twitter-airline-sentiment/Tweets.csv")

In [3]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


Colunas de interesse: id, sentimento (label) e o texto em si

In [6]:
train_ds = df[['tweet_id', 'airline_sentiment', 'text']]
train_ds.head()

,tweet_id,airline_sentiment,text
0,570306133677760513,neutral,@VirginAmerica What @dhepburn said.
1,570301130888122368,positive,@VirginAmerica plus you've added commercials t...
2,570301083672813571,neutral,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,negative,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,negative,@VirginAmerica and it's a really big bad thing...


In [8]:
train_ds['airline_sentiment'] = train_ds['airline_sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0})
train_ds.head()

,tweet_id,airline_sentiment,text
0,570306133677760513,1,@VirginAmerica What @dhepburn said.
1,570301130888122368,2,@VirginAmerica plus you've added commercials t...
2,570301083672813571,1,@VirginAmerica I didn't today... Must mean I n...
3,570301031407624196,0,@VirginAmerica it's really aggressive to blast...
4,570300817074462722,0,@VirginAmerica and it's a really big bad thing...


Vamos usar o sklearn para extrair features do texto.

A maneira mais simples de fazer isso é utilizar a contagem de frequência de palavras no texto

In [12]:
count_vectorizer = CountVectorizer(max_features=5000)

feature_vector = count_vectorizer.fit(train_ds.text)
train_ds_features = count_vectorizer.transform(train_ds.text)

train_ds_features

<14640x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 222797 stored elements in Compressed Sparse Row format>

In [15]:
features = feature_vector.get_feature_names()
features[1000:2000]

['coincidence',
 'coke',
 'cold',
 'colleague',
 'colleagues',
 'collect',
 'collection',
 'college',
 'color',
 'columbia',
 'columbus',
 'com',
 'combination',
 'combine',
 'comcast',
 'come',
 'comedy',
 'comes',
 'comfort',
 'comfortable',
 'coming',
 'comm',
 'comment',
 'comments',
 'commercial',
 'commercials',
 'commitment',
 'committed',
 'common',
 'communicate',
 'communicated',
 'communication',
 'communications',
 'community',
 'comp',
 'companies',
 'companion',
 'company',
 'compared',
 'compassion',
 'comped',
 'compensate',
 'compensated',
 'compensation',
 'competent',
 'competition',
 'competitor',
 'complain',
 'complained',
 'complaint',
 'complaints',
 'complete',
 'completed',
 'completely',
 'complicated',
 'compliment',
 'complimentary',
 'computer',
 'computers',
 'con',
 'concept',
 'concern',
 'concerned',
 'concerns',
 'concert',
 'concourse',
 'condition',
 'conditions',
 'conf',
 'conference',
 'confidence',
 'confident',
 'confirm',
 'confirmation',
 'co

In [16]:
features_counts = np.sum( train_ds_features.toarray(), axis = 0 )
feature_counts = pd.DataFrame( dict( features = features,
                                  counts = features_counts ) )

feature_counts.head(5)


,features,counts
0,00,14
1,000,31
2,0016,3
3,00pm,5
4,02,6


Vamos remover stopwords

In [17]:
count_vectorizer = CountVectorizer( stop_words = "english",
                                 max_features = 5000 )
feature_vector = count_vectorizer.fit( train_ds.text )
train_ds_features = count_vectorizer.transform( train_ds.text )

features = feature_vector.get_feature_names()
features_counts = np.sum( train_ds_features.toarray(), axis = 0 )
feature_counts = pd.DataFrame( dict( features = features,
                                  counts = features_counts ) )
feature_counts.sort_values( "counts", ascending = False )[0:20]

,features,counts
4708,united,4164
1919,flight,3939
4746,usairways,3053
493,americanair,2964
4230,southwestair,2461
2617,jetblue,2395
2425,http,1155
4472,thanks,1083
996,cancelled,1065
2646,just,974


In [18]:
train_ds_features

<14640x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 122964 stored elements in Compressed Sparse Row format>

In [143]:
transformer = TfidfTransformer()
tf_idf_vector = transformer.fit(train_ds_features)
tf_idf_features = transformer.fit_transform(train_ds_features)
tf_idf_features.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Metricas

Vamos utilizar como metricas do clustering: homogeinity e completeness

Vamos comparar os resultados do spectral clustering com os resultados obtidos utilizando o kmeans++

In [138]:
kmeans = cluster.KMeans(n_clusters=3)
k_means_preds = kmeans.fit_predict(tf_idf_features)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=k_means_preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=k_means_preds)
print("Completeness: {} \nHomogeinity: {}".format(completeness, homogeinity))

Completeness: 0.016127271008612595 
Homogeinity: 0.01676096076673386


### Métodos de spectral clustering

Na célula abaixamos implementamos os três diferentes métodos propostos no artigo.

A seguir vamos construir os grafos de similaridade e testar cada um dos métodos, variando o tipo de grafo e o tipo de função de similaridade

In [126]:
def unnormalized_spectral_clustering(similarity_matrix, num_clusters):
    G = nx.from_scipy_sparse_matrix(similarity_matrix)
    laplacian = nx.laplacian_matrix(G)
    eigvals, eigvecs = scipy.sparse.linalg.eigs(laplacian.toarray(), k=num_clusters, which='SM')
    eigvecs = np.real_if_close(eigvecs)
    kmeans = cluster.KMeans(n_clusters=num_clusters)
    preds = kmeans.fit_predict(eigvecs)
    return preds

def shi_spectral_clustering(similarity_matrix, num_clusters):
    G = nx.from_scipy_sparse_matrix(similarity_matrix)
    laplacian = nx.laplacian_matrix(G)
    n, m = similarity_matrix.shape
    diags = similarity_matrix.sum(axis=1)
    D = scipy.sparse.spdiags(diags.flatten(), [0], m, n, format='csr')
    eigvals, eigvecs = scipy.sparse.linalg.eigs(laplacian.toarray(), k=num_clusters, which='SM', M=D)
    eigvecs = np.real_if_close(eigvecs)
    kmeans = cluster.KMeans(n_clusters=num_clusters)
    preds = kmeans.fit_predict(eigvecs)
    return preds

def ng_spectral_clustering(similarity_matrix, num_clusters):
    G = nx.from_scipy_sparse_matrix(similarity_matrix)
    laplacian = nx.normalized_laplacian_matrix(G)
    n, m = similarity_matrix.shape
    diags = similarity_matrix.sum(axis=1)
    D = scipy.sparse.spdiags(diags.flatten(), [0], m, n, format='csr')
    eigvals, eigvecs = scipy.sparse.linalg.eigs(laplacian.toarray(), k=num_clusters, which='SM', M=D)
    eigvecs = np.real_if_close(eigvecs)
    T = eigvecs / np.linalg.norm(eigvecs, axis=1)[:, np.newaxis]
    kmeans = cluster.KMeans(n_clusters=num_clusters)
    preds = kmeans.fit_predict(eigvecs)
    return preds

### Grafo de similaridade

Primeiro, tentamos com a distancia euclidiana, por simplicidade

### KNN graph

Os autores sugerem que o numero de vizinhos deve ser suficientemente grande para que tenhamos um grafo conexo

Vale ressaltar que para valores pequenos no numero de vizinhos as matrizes D ficaram mal-condicionadas, tornando o problema de encontrar os autovetores generalizados inviavel

Diversos valores foram testados para gerar o knn_graph, verificou-se que para os dois primeiros algoritmos, um numero maior de vizinhos teve um resultado melhor, enquanto que para o terceiro algoritmo n=50 foi o que teve o melhor resultado. Note que ele foi o maior dentre os tres, utilizando a distancia euclidiana como metrica

In [183]:
knn_graph = kneighbors_graph(tf_idf_features, n_neighbors=200, mode='distance', include_self=True)
G = nx.from_scipy_sparse_matrix(knn_graph)
print([len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)])

[14640]


### Unnormalized spectral clustering

In [184]:
preds = unnormalized_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.18775390710410184 
 Homogeinity: 0.1296837698851961


### Shi Spectral Clustering

In [185]:
preds = shi_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.1819380134580632 
 Homogeinity: 0.13504716013787285


### Ng Spectral Clustering

In [181]:
knn_graph = kneighbors_graph(tf_idf_features, n_neighbors=50, mode='distance', include_self=True)
preds = ng_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.17662664761262414 
 Homogeinity: 0.1342070759529893


Os resultados obtidos foram melhores do que os obtidos com kmeans, mas os valores das metricas ainda estao baixos.

Vamos testar cosine distance como metrica de distancia/similaridade, uma vez que é bastante utilizada ao comparar vetores de frequencia de palavras

## Metrica de distancia: cosine distance

Diversos valores para o numero de vizinhos foram testado n=100 foi o que obteve o melhor resultado

In [168]:
knn_graph = kneighbors_graph(tf_idf_features, n_neighbors=200, mode='distance', include_self=True, metric='cosine')

[14640]


### Unnormalized spectral clustering

In [169]:
preds = unnormalized_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.18304064106374418 
 Homogeinity: 0.12018453196332368


### Shi Spectral Clustering

In [170]:
preds = shi_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.17744736102618325 
 Homogeinity: 0.12821335249492244


### Ng Spectral Clustering

In [182]:
knn_graph = kneighbors_graph(tf_idf_features, n_neighbors=50, mode='distance', include_self=True, metric='cosine')
preds = ng_spectral_clustering(similarity_matrix=knn_graph, num_clusters=3)
completeness = metrics.completeness_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
homogeinity = metrics.homogeneity_score(labels_true=train_ds.airline_sentiment, labels_pred=preds)
print("Completeness: {} \n Homogeinity: {}".format(completeness, homogeinity))

Completeness: 0.14522097089203506 
 Homogeinity: 0.11569449084545855


## Fully connected graph

Vamos realizar os mesmos testes utilizando um grafo fully connected

Importante notar que, apesar de mais simples em termos de hiperparâmetros, o tempo de execução e requisito de memória para esse tipo de grafo é bem maior

In [ ]:
fc_graph = metrics.pairwise.pairwise_distances(tf_idf_features.toarray(), metric='minkowski')